修改表有以下动作：
- 增加列
- 删除列
- 修改列（改变类型）
- 为列增加默认值
- 为列增加 constraint
- 重命名表

```
ALTER TABLE table_name 
ADD COLUMN column_name datatype column_constraint;
```
```
ALTER TABLE table_name 
DROP COLUMN column_name;
```
```
ALTER TABLE table_name 
RENAME COLUMN column_name 
TO new_column_name;
```
```
ALTER TABLE table_name 
ALTER COLUMN column_name 
[SET DEFAULT value | DROP DEFAULT];
```
```
ALTER TABLE table_name 
ALTER COLUMN column_name 
[SET NOT NULL| DROP NOT NULL];
```
```
ALTER TABLE table_name 
ADD CHECK expression;
```
```
ALTER TABLE table_name 
ADD CONSTRAINT constraint_name constraint_definition;
```
```
ALTER TABLE table_name 
RENAME TO new_table_name;
```

In [1]:
import common.ipynb_importer
from db.pg.pg_00_common import *

cursor = pg_connect()

importing Jupyter notebook from E:\sourcecode\keep_learning\db\pg\pg_00_common.ipynb


In [2]:
# add column
sql = """
DROP TABLE IF EXISTS links;

CREATE TABLE links (
   link_id serial PRIMARY KEY,
   title VARCHAR (512) NOT NULL,
   url VARCHAR (1024) NOT NULL
);
"""
cursor.execute(sql)

sql = """
SELECT *
FROM information_schema.columns
WHERE table_name = 'links';
"""
run_sql(cursor, sql)

  table_catalog table_schema table_name column_name  ordinal_position  \
0     dvdrental       public      links     link_id                 1   
1     dvdrental       public      links       title                 2   
2     dvdrental       public      links         url                 3   

                           column_default is_nullable          data_type  \
0  nextval('links_link_id_seq'::regclass)          NO            integer   
1                                    None          NO  character varying   
2                                    None          NO  character varying   

   character_maximum_length  character_octet_length  ...  is_identity  \
0                       NaN                     NaN  ...           NO   
1                     512.0                  2048.0  ...           NO   
2                    1024.0                  4096.0  ...           NO   

   identity_generation  identity_start identity_increment identity_maximum  \
0                 None         

In [3]:
sql = """
ALTER TABLE links
ADD COLUMN active boolean;
"""
cursor.execute(sql)

sql = """
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name = 'links';
"""
run_sql(cursor, sql)

  column_name          data_type
0     link_id            integer
1       title  character varying
2         url  character varying
3      active            boolean


In [4]:
sql = """
ALTER TABLE links 
DROP COLUMN active;
"""
cursor.execute(sql)

sql = """
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name = 'links';
"""
run_sql(cursor, sql)

  column_name          data_type
0     link_id            integer
1       title  character varying
2         url  character varying


In [5]:
sql = """
ALTER TABLE links 
RENAME COLUMN title TO link_title;
"""
cursor.execute(sql)

sql = """
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name = 'links';
"""
run_sql(cursor, sql)

  column_name          data_type
0     link_id            integer
1  link_title  character varying
2         url  character varying


In [6]:
sql = """
ALTER TABLE links 
ADD COLUMN target VARCHAR(10);
"""
cursor.execute(sql)

sql = """
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name = 'links';
"""
run_sql(cursor, sql)

  column_name          data_type
0     link_id            integer
1  link_title  character varying
2         url  character varying
3      target  character varying


In [7]:
sql = """
ALTER TABLE links 
ALTER COLUMN target
SET DEFAULT '_blank';
"""
cursor.execute(sql)

sql = """
SELECT column_name, data_type, column_default
FROM information_schema.columns
WHERE table_name = 'links';
"""
run_sql(cursor, sql)

  column_name          data_type                          column_default
0     link_id            integer  nextval('links_link_id_seq'::regclass)
1  link_title  character varying                                    None
2         url  character varying                                    None
3      target  character varying             '_blank'::character varying


In [8]:
sql = """
INSERT INTO links (link_title, url)
VALUES('PostgreSQL Tutorial','https://www.postgresqltutorial.com/');
"""
cursor.execute(sql)

sql = """
SELECT *
FROM links;
"""
run_sql(cursor, sql)

   link_id           link_title                                  url  target
0        1  PostgreSQL Tutorial  https://www.postgresqltutorial.com/  _blank


In [9]:
sql = """
ALTER TABLE links 
ADD CHECK (target IN ('_self', '_blank', '_parent', '_top'));
"""
cursor.execute(sql)

sql = """
SELECT *
FROM information_schema.columns
WHERE table_name = 'links';
"""
run_sql(cursor, sql)

  table_catalog table_schema table_name column_name  ordinal_position  \
0     dvdrental       public      links     link_id                 1   
1     dvdrental       public      links  link_title                 2   
2     dvdrental       public      links         url                 3   
3     dvdrental       public      links      target                 5   

                           column_default is_nullable          data_type  \
0  nextval('links_link_id_seq'::regclass)          NO            integer   
1                                    None          NO  character varying   
2                                    None          NO  character varying   
3             '_blank'::character varying         YES  character varying   

   character_maximum_length  character_octet_length  ...  is_identity  \
0                       NaN                     NaN  ...           NO   
1                     512.0                  2048.0  ...           NO   
2                    1024.0       

In [10]:
sql = """
INSERT INTO links(link_title,url,target) 
VALUES('PostgreSQL','http://www.postgresql.org/','whatever');
"""
cursor.execute(sql)

sql = """
SELECT *
FROM links;
"""
run_sql(cursor, sql)

CheckViolation: 关系 "links" 的新列违反了检查约束 "links_target_check"
DETAIL:  失败, 行包含(2, PostgreSQL, http://www.postgresql.org/, whatever).

In [ ]:
sql = """
ALTER TABLE links 
ADD CONSTRAINT unique_url UNIQUE ( url );
"""
cursor.execute(sql)

sql = """
SELECT column_name, data_type, column_default
FROM information_schema.columns
WHERE table_name = 'links';
"""
run_sql(cursor, sql)